In [7]:
%matplotlib inline
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from scipy import stats
#import powerlaw 
import seaborn as sns

In [8]:
train = pd.read_csv('we/dataset/train.csv')
test = pd.read_csv('we/dataset/test.csv')
valid = pd.read_csv('we/dataset/validation.csv')

In [9]:
import sqlite3
from math import sqrt
from sklearn.feature_extraction import DictVectorizer as DV
#from sklearn.svm import SVC
#from sklearn.ensemble import RandomForestClassifier as RF
#from sklearn.metrics import roc_auc_score as AUC
import statsmodels.api as sm


In [10]:
train["area"]=train["slotheight"]*train["slotwidth"]
test["area"]=test["slotheight"]*test["slotwidth"]

In [11]:
#train_cols = data1.columns[1:]
from sklearn import preprocessing
import statsmodels.api as sm
#from sklearn.linear_model import LogisticRegression
#from sklearn.ensemble import RandomForestRegressor, BaggingRegressor
label_encoder = preprocessing.LabelEncoder()
train['hour']=label_encoder.fit_transform(train['hour'])
train['weekday']=label_encoder.fit_transform(train['weekday'])
train['advertiser']=label_encoder.fit_transform(train['advertiser'])
train['area']=label_encoder.fit_transform(train['area'])
train['adexchange']=label_encoder.fit_transform(train['adexchange'])
train['slotprice']=label_encoder.fit_transform(train['slotprice'])
train['slotformat']=label_encoder.fit_transform(train['slotformat'])
train['slotvisibility']=label_encoder.fit_transform(train['slotvisibility'])

features =['weekday', 'hour','advertiser','area','adexchange','slotprice','slotformat','slotvisibility']

test['hour']=label_encoder.fit_transform(test['hour'])
test['weekday']=label_encoder.fit_transform(test['weekday'])
test['advertiser']=label_encoder.fit_transform(test['advertiser'])
test['area']=label_encoder.fit_transform(test['area'])
test['adexchange']=label_encoder.fit_transform(test['adexchange'])
test['slotprice']=label_encoder.fit_transform(test['slotprice'])
test['slotformat']=label_encoder.fit_transform(test['slotformat'])
test['slotvisibility']=label_encoder.fit_transform(test['slotvisibility'])

In [12]:
from sklearn import ensemble
#from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier

In [13]:
x_train=train[features]
y_train=train['click']
x_test=test[features]

In [14]:
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
#rf = RandomForestRegressor(n_estimators=15, max_depth=6, random_state=0)
rf = RandomForestClassifier(n_estimators=20, max_depth=7, random_state=0)
clf = BaggingClassifier(rf, n_estimators=40, max_samples=0.1, random_state=25)
clf.fit(x_train, y_train)
y_pred2 = clf.predict_proba(x_test)
y_pred2

array([[  9.98953182e-01,   1.04681798e-03],
       [  9.99000528e-01,   9.99471828e-04],
       [  9.99562844e-01,   4.37156018e-04],
       ..., 
       [  9.98799170e-01,   1.20083033e-03],
       [  9.99632907e-01,   3.67093199e-04],
       [  9.99723234e-01,   2.76765856e-04]])

In [15]:

params = {'n_estimators': 250, 'max_depth': 4, 'min_samples_split': 2,
          'learning_rate': 0.05, 'loss': 'deviance'}
clf = ensemble.GradientBoostingClassifier(**params)

clf.fit(x_train, y_train)

y_pred1 = clf.predict_proba(x_test)
y_pred1

array([[  9.98762150e-01,   1.23785027e-03],
       [  9.99311494e-01,   6.88505770e-04],
       [  9.99668452e-01,   3.31548163e-04],
       ..., 
       [  9.98767737e-01,   1.23226324e-03],
       [  9.99581179e-01,   4.18821193e-04],
       [  9.99692159e-01,   3.07841333e-04]])

In [16]:
pre_1=[]
for a in y_pred1:
    pre_1.append(a[1])
len(pre_1)

299749

In [17]:
pre_2=[]
for b in y_pred2:
    pre_2.append(b[1])
len(pre_2)

299749

In [165]:
#y_pre=valid['click']

In [18]:
pre_3=[]
for i in range(0,299749):
    pre=0.9*pre_1[i]+0.1*pre_2[i]
    pre_3.append(pre)

In [19]:
#from sklearn.metrics import mean_squared_error, make_scorer
#mse = mean_squared_error(y_pre, pre_3, sample_weight=None, multioutput='uniform_average')
#np.sqrt(mse)

In [39]:
#import numpy as np
#from sklearn.metrics import roc_auc_score
#auc=roc_auc_score(y_pre, pre_3)
#auc
#train[:10]

In [33]:
avg_CTR=sum(train['click'])/len(train)
avg_CTR

0.0007539649884458758

In [34]:
#base_bid=sum(train['bidprice'])/len(train)
#from statistics import mode
base_bid=70 #average bid, 150

In [35]:
#L1=valid['bidprice'].values/1000
#L2=valid['payprice'].values/1000

In [36]:
L=[]

for i in range(0,299749):
    p_bid=base_bid*(pre_3[i])/avg_CTR
    L.append(p_bid)

In [37]:
#L
final = pd.DataFrame({"bidid": test['bidid'], "bidprice": L})

In [52]:
#final[-1:]
#test[-1:]

In [45]:
final.to_csv('COMPGW02.csv',index=False)